# 6. Purely functional state

## 6.1 Generating random numbers using side effects

In [1]:
val rng=new scala.util.Random


scala.util.Random@b7e142

In [2]:
rng.nextDouble

0.8537923442917561

In [3]:
rng.nextDouble

0.03818834905616353

In [4]:
rng.nextInt

975753803

In [5]:
def rollingDie: Int = {
    val rng = new scala.util.Random
    rng.nextInt(6)
}

In [6]:
rollingDie

3

In [7]:
rollingDie

2

In [8]:
rollingDie

0

### => all using internal state variable => SIDE EFFECT!

## 6.2 Purely functional random number generation

In [9]:
trait RNG {
    def nextInt: (Int, RNG)
}

### => 난수 계산과 state 전달 분리.

In [10]:
case class SimpleRNG(seed: Long) extends RNG {
    def nextInt: (Int, RNG) = {
        val newSeed = (seed * 0x5DEECE66DL + 0xBL) & 0xFFFFFFFFFFFFL
        val nextRNG = SimpleRNG(newSeed)
        val n = (newSeed >>> 16).toInt
        (n, nextRNG)
    }
}

In [11]:
val rng = SimpleRNG(42)

SimpleRNG(42)

In [12]:
val (n1, rng2) = rng.nextInt

16159453

In [13]:
val (n2, rng3) = rng2.nextInt

-1281479697

In [76]:
rng2

SimpleRNG(1059025964525)

In [77]:
rng3

SimpleRNG(197491923327988)

## 6.3 Making stateful APIs pure

In [17]:
def randomPair0(rng:RNG):(Int, Int) = {
    val (i1, _) = rng.nextInt
    val (i2, _) = rng.nextInt
    (i1,i2)
}

In [18]:
randomPair0(rng2)

(-1281479697,-1281479697)

In [19]:
def randomPair(rng:RNG): ((Int,Int), RNG) = {
    val (i1, rng2) = rng.nextInt
    val (i2, rng3) = rng2.nextInt
    ((i1,i2),rng3)
}

In [20]:
randomPair(rng2)

((-1281479697,-340305902),SimpleRNG(259172689157871))

### Ex.6.1 nonNegativeInt [0 ~ Int.maxValue]

In [21]:
def nonNegativeInt(rng:RNG): (Int, RNG) = {
    var n = 0
    val (i, rngNext) = rng.nextInt
    if (i==Int.MinValue) n = - (Int.MinValue+1)
    if (i<0) n = -i
    else n = i
    (n,rngNext)
// book's answer
// if (i < 0) (-(i+1),rngNext) else (i, rngNext) 
}

In [22]:
val (n1, r3) = nonNegativeInt(rng2)

1281479697

In [23]:
r3

SimpleRNG(197491923327988)

In [24]:
nonNegativeInt(r3)

(340305902,SimpleRNG(259172689157871))

### Ex.6.2 double

In [25]:
def double(rng:RNG): (Double, RNG) = {
    val (i,r) = nonNegativeInt(rng)
    if (i!=Int.MaxValue) ((i.toDouble / Int.MaxValue.toDouble), r)
    else ((Int.MaxValue-1).toDouble / Int.MaxValue.toDouble, r)
/*** book's answer
def double(rng: RNG): (Double, RNG) = {
  val (i, r) = nonNegativeInt(rng)
  (i / (Int.MaxValue.toDouble + 1), r)
}
****/
}

In [26]:
double(rng2)

(0.5967354856416283,SimpleRNG(197491923327988))

In [27]:
def intDouble(rng:RNG): ((Int,Double), RNG) = {
    val (i, r1) = rng.nextInt
    val (d, r2) = double(r1)
    ((i,d), r2)
}

In [28]:
intDouble(rng2)

((-1281479697,0.15846728447753344),SimpleRNG(259172689157871))

### Ex.6.3 doubleInt, intDouble, double3

In [29]:
def doubleInt(rng:RNG): ((Double,Int), RNG) = {
    val (i, r1) = rng.nextInt
    val (d, r2) = double(r1)
    ((d, i), r2)
}

In [30]:
doubleInt(rng2)

((0.15846728447753344,-1281479697),SimpleRNG(259172689157871))

In [31]:
def double3(rng:RNG): ((Double,Double,Double),RNG) = {
    val (d1, r1) = double(rng)
    val (d2, r2) = double(r1)
    val (d3, r3) = double(r2)
    ((d1,d2,d3),r3)
}

In [32]:
double3(rng2)

((0.5967354856416283,0.15846728447753344,0.9386595436086224),SimpleRNG(149370390209998))

### Ex.6.4 ints

In [33]:
def ints(count:Int)(rng:RNG): (List[Int], RNG) = {
    def go(n:Int,rl:List[Int],rngIn:RNG): (List[Int],RNG) = {
        if (n==0) (rl, rngIn)
        else {
            val (i,rngNext) = rngIn.nextInt
            go(n-1, rl :+ i, rngNext)
        }
    }
    go(count,List(),rng)
}

In [34]:
ints(5)(rng2)

(List(-1281479697, -340305902, -2015756020, 1770001318, -1934589059),SimpleRNG(154689748186168))

> As you write more functional programs, you’ll sometimes encounter situations like
this where the functional way of expressing a program feels awkward or tedious. Does
this imply that purity is the equivalent of trying to write an entire novel without using
the letter E? Of course not. ** Awkwardness like this is almost always a sign of some
missing abstraction waiting to be discovered.**

## 6.4 A better API for state actions

### common pattern("state actions", "state transitions"):   
    RNG => (A, RNG)

### RNG state action data type:

In [35]:
type Rand[+A] = RNG => (A, RNG)

#### nextInt as a type of Rand[Int]

In [36]:
val int: Rand[Int] = _.nextInt

<function1>

### combinators - unit (do nothing)

In [37]:
def unit[A](a:A): Rand[A] = 
    rng => (a,rng)

### combinators - map (function composition)

In [38]:
def map[A,B](s:Rand[A])(f: A => B): Rand[B] = 
    rng => {
        val (a, rng2) = s(rng)
        (f(a), rng2)
    }

#### using map to implement nonNegativeEven

In [39]:
def nonNegativeEven: Rand[Int] =
    map(nonNegativeInt)(i => i - i % 2)

In [40]:
nonNegativeEven(rng2)

(1281479696,SimpleRNG(197491923327988))

In [41]:
nonNegativeEven(rng3)

(340305902,SimpleRNG(259172689157871))

### Ex.6.5 double(using 'map'. cf. Ex.6.2)

In [42]:
def convToDouble(i:Int): Double = {
    val maxFloat = Int.MaxValue.toDouble
    if (i!=Int.MaxValue) i.toDouble / maxFloat
    else (i-1).toDouble / maxFloat
}
    
def doubleViaMap: Rand[Double] = 
    map(nonNegativeInt)(i => convToDouble(i))

In [43]:
doubleViaMap(rng2)

(0.5967354856416283,SimpleRNG(197491923327988))

In [44]:
double(rng2)

(0.5967354856416283,SimpleRNG(197491923327988))

### 6.4.1 Combining state actions

In [45]:
def map2[A,B,C](ra:Rand[A], rb:Rand[B])(f: (A,B) => C): Rand[C] = {
    rng => {
        val (a, rngA) = ra(rng)
        val (b, rngB) = rb(rngA)
        (f(a,b), rngB)
    }
}

In [46]:
def both[A,B](ra:Rand[A], rb:Rand[B]): Rand[(A,B)] = map2(ra,rb)((_, _))

In [47]:
val randIntDouble: Rand[(Int,Double)] = both(int,double)

<function1>

In [48]:
val randDoubleInt: Rand[(Double,Int)] = both(double, int)

<function1>

In [49]:
intDouble(rng2)

((-1281479697,0.15846728447753344),SimpleRNG(259172689157871))

In [50]:
randIntDouble(rng2)

((-1281479697,0.15846728447753344),SimpleRNG(259172689157871))

### Ex.6.7 sequence

In [51]:
def sequence[A](fs: List[Rand[A]]): Rand[List[A]] = {
    def go(ss: List[Rand[A]], ll:List[A], rngIn:RNG): (List[A], RNG) = ss match {
        case h :: t => {
            val (a, rngOut) = h(rngIn)
            go(t, ll :+ a, rngOut)
        }
        case _ => (ll, rngIn)
    }
    rng => go(fs, List(), rng)
/*
book's answer:
def sequence ... 
    fs.foldRight(unit(List[A]()))((f,acc) => map2(f,acc)(_ :: _))

def _ints(count:Int): Rand[List[Int]] = 
    sequence(List.fill(count)(int))
*/
}

In [52]:
def randInts(count:Int)(rng:RNG): (List[Int], RNG) = {
    sequence(List.fill(count)(int))(rng)
}

In [53]:
ints(5)(rng2)

(List(-1281479697, -340305902, -2015756020, 1770001318, -1934589059),SimpleRNG(154689748186168))

In [54]:
randInts(5)(rng2)

(List(-1281479697, -340305902, -2015756020, 1770001318, -1934589059),SimpleRNG(154689748186168))

### 6.4.2 Nesting state actions

In [55]:
def nonNegativeLessThan_1st_try(n:Int): Rand[Int] = 
    map(nonNegativeInt)(_ % n)

#### => not fair (or 'skewed') (Why?? cf. https://forums.manning.com/posts/list/34423.page)

In [56]:
def tryN(count:Int)(f: Rand[Int]): Rand[List[Int]] = sequence(List.fill(count)(f))

In [57]:
tryN(100)(nonNegativeLessThan_1st_try(100))(rng2)

(List(97, 2, 20, 18, 59, 12, 41, 59, 74, 94, 32, 17, 91, 81, 21, 62, 33, 77, 70, 0, 36, 48, 92, 59, 64, 63, 11, 99, 16, 76, 22, 87, 43, 24, 70, 2, 47, 39, 68, 22, 36, 2, 64, 78, 85, 22, 39, 69, 46, 39, 4, 4, 27, 36, 18, 11, 16, 36, 8, 9, 26, 74, 25, 94, 54, 52, 18, 53, 60, 22, 69, 49, 91, 2, 4, 0, 97, 19, 29, 98, 32, 55, 48, 6, 43, 19, 75, 84, 64, 42, 68, 13, 82, 35, 9, 20, 29, 6, 13, 60),SimpleRNG(186499461027745))

In [94]:
tryN(100)(nonNegativeLessThan_1st_try(100))(rng2)._1.filter(_ < 50).length

60

In [95]:
def nonNegativeLessThan_2nd(n: Int): Rand[Int] = { rng =>
    val (i, rng2) = nonNegativeInt(rng)
    val mod = i % n
    if (i + (n-1) - mod >= 0)
        (mod, rng2)
    else nonNegativeLessThan_2nd(n)(rng2)
}

In [96]:
tryN(1000)(nonNegativeLessThan_1st_try(100))(SimpleRNG(11163)) == 
tryN(1000)(nonNegativeLessThan_2nd(100))(SimpleRNG(11163))

true

### Ex. 6.8 flatMap

In [98]:
def flatMap[A,B](f: Rand[A])(g: A => Rand[B]): Rand[B] = {
    rng => {
        val (i,rngNext) = f(rng)
        g(i)(rngNext)    // g(i) ??
    }
}

In [62]:
def nonNegativeLessThan(n:Int): Rand[Int] = {
    flatMap(nonNegativeInt){ i =>
        val mod = i % n
        if (i + (n-1) - mod >= 0) unit(mod) else nonNegativeLessThan(n)
    }
}

In [97]:
tryN(100)(nonNegativeLessThan(100))(rng2)._2

SimpleRNG(186499461027745)

### Ex. 6.9 mapViaFlatMap, map2ViaFlatMap

In [64]:
def mapViaFlatMap[A,B](s: Rand[A])(f: A => B): Rand[B] = {
    /* flatMap(s)(a => (f(a),_)) */
    flatMap(s)(a => unit(f(a)))
}

In [65]:
def _nonNegativeEven: Rand[Int] =
    mapViaFlatMap(nonNegativeInt)(i => i - i % 2)

In [66]:
nonNegativeEven(rng2)

(1281479696,SimpleRNG(197491923327988))

In [67]:
_nonNegativeEven(rng2)

(1281479696,SimpleRNG(197491923327988))

In [68]:
def map2ViaFlatMap[A,B,C](ra: Rand[A], rb:Rand[B])(f: (A,B) => C): Rand[C] = {
   flatMap(ra)(a => map(rb)(b => f(a,b)))
}

In [69]:
def _both[A,B](ra:Rand[A], rb:Rand[B]): Rand[(A,B)] = {
    map2ViaFlatMap(ra,rb)((_, _))
    /* def both[A,B](ra:Rand[A], rb:Rand[B]): Rand[(A,B)] = map2(ra,rb)((_, _)) */
}
val _randIntDouble: Rand[(Int,Double)] = _both(int,double)

<function1>

In [70]:
randIntDouble(rng2)

((-1281479697,0.15846728447753344),SimpleRNG(259172689157871))

In [71]:
_randIntDouble(rng2)

((-1281479697,0.15846728447753344),SimpleRNG(259172689157871))

In [72]:
def rollDie: Rand[Int] = nonNegativeLessThan(6)

In [73]:
tryN(100)(rollDie)(rng2)

(List(3, 2, 4, 4, 5, 2, 1, 5, 2, 4, 4, 1, 3, 1, 3, 4, 3, 3, 4, 0, 4, 0, 2, 1, 2, 3, 5, 5, 0, 2, 4, 5, 5, 2, 4, 4, 3, 1, 2, 2, 2, 2, 4, 2, 5, 4, 3, 1, 4, 1, 4, 4, 1, 0, 4, 5, 2, 0, 2, 5, 4, 4, 3, 2, 2, 0, 0, 3, 2, 0, 3, 1, 1, 2, 2, 4, 3, 1, 3, 0, 2, 3, 2, 2, 3, 1, 1, 4, 2, 4, 2, 5, 4, 5, 5, 2, 1, 4, 3, 4),SimpleRNG(186499461027745))

In [74]:
def _rollDie: Rand[Int] = map(nonNegativeLessThan(6))(_ + 1)

In [75]:
tryN(100)(_rollDie)(rng2)

(List(4, 3, 5, 5, 6, 3, 2, 6, 3, 5, 5, 2, 4, 2, 4, 5, 4, 4, 5, 1, 5, 1, 3, 2, 3, 4, 6, 6, 1, 3, 5, 6, 6, 3, 5, 5, 4, 2, 3, 3, 3, 3, 5, 3, 6, 5, 4, 2, 5, 2, 5, 5, 2, 1, 5, 6, 3, 1, 3, 6, 5, 5, 4, 3, 3, 1, 1, 4, 3, 1, 4, 2, 2, 3, 3, 5, 4, 2, 4, 1, 3, 4, 3, 3, 4, 2, 2, 5, 3, 5, 3, 6, 5, 6, 6, 3, 2, 5, 4, 5),SimpleRNG(186499461027745))

## 6.5 A general state action data type

### - NOTE: map() 함수 자체는 RNG를 다루는지조차 모름.

In [115]:
/*
def map[S,A,B](a: S => (A,S))(f: A => B): S => (B,S) ={
    S => {
        val (v, nextState) = a(S)
        (f(v), nextState)
    }
    // def map[A,B](s:Rand[A])(f: A => B): Rand[B] = 
    // rng => {
    //     val (a, rng2) = s(rng)
    //     (f(a), rng2)
    // }
}
*/

: 

### - map(), map2(), flatmap()을 (state action을 다룰 수 있는) 일반적인 함수로 만들어보자!

In [117]:
/* type State[S,+A] = S => (A, S) */

: 

In [114]:
/* case class State[S,+A](run: S => (A,S)) */

: 

#### Rand[A]는 State[S,+A]의 인스턴스 중 하나로 볼 수 있음.

In [116]:
// type Rand[A] = State[RNG, A]

: 

### Ex. 6.10 generalize map(),map2(),flatMap() and sequence() as functions on the State class

#### 책의 답안이 컴파일이 안 돼....ㅜ.ㅜ

In [120]:
case class State[S, +A](run: S => (A, S)) {
  def map[B](f: A => B): State[S, B] =
    flatMap(a => unit(f(a)))
  def map2[B,C](sb: State[S, B])(f: (A, B) => C): State[S, C] =
    flatMap(a => sb.map(b => f(a, b)))
  def flatMap[B](f: A => State[S, B]): State[S, B] = State(s => {
    val (a, s1) = run(s)
    f(a).run(s1)
  })
}

object State {

  def unit[S, A](a: A): State[S, A] =
    State(s => (a, s))

  // The idiomatic solution is expressed via foldRight
  def sequenceViaFoldRight[S,A](sas: List[State[S, A]]): State[S, List[A]] =
    sas.foldRight(unit[S, List[A]](List()))((f, acc) => f.map2(acc)(_ :: _))

  // This implementation uses a loop internally and is the same recursion
  // pattern as a left fold. It is quite common with left folds to build
  // up a list in reverse order, then reverse it at the end.
  // (We could also use a collection.mutable.ListBuffer internally.)
  def sequence[S, A](sas: List[State[S, A]]): State[S, List[A]] = {
    def go(s: S, actions: List[State[S,A]], acc: List[A]): (List[A],S) =
      actions match {
        case Nil => (acc.reverse,s)
        case h :: t => h.run(s) match { case (a,s2) => go(s2, t, a :: acc) }
      }
    State((s: S) => go(s,sas,List()))
  }

  // We can also write the loop using a left fold. This is tail recursive like the
  // previous solution, but it reverses the list _before_ folding it instead of after.
  // You might think that this is slower than the `foldRight` solution since it
  // walks over the list twice, but it's actually faster! The `foldRight` solution
  // technically has to also walk the list twice, since it has to unravel the call
  // stack, not being tail recursive. And the call stack will be as tall as the list
  // is long.
  def sequenceViaFoldLeft[S,A](l: List[State[S, A]]): State[S, List[A]] =
    l.reverse.foldLeft(unit[S, List[A]](List()))((acc, f) => f.map2(acc)( _ :: _ ))

  def modify[S](f: S => S): State[S, Unit] = for {
    s <- get // Gets the current state and assigns it to `s`.
    _ <- set(f(s)) // Sets the new state to `f` applied to `s`.
  } yield ()

  def get[S]: State[S, S] = State(s => (s, s))

  def set[S](s: S): State[S, Unit] = State(_ => ((), s))
}

: 

## 6.6 Purely functional imperative programming

### FP는 단순히 얘기하면 "side-effect 없는 프로그래밍 기법"

#### functional style

In [105]:
/* val ns: Rand[List[Int]] = 
    int.flatMap(x =>
               int.flatMap(y =>
                          ints(x).map(xs =>
                                     xs.map(_ % y))))
                                     */

: 

#### imperative style  (but /wo side-effects)

In [106]:
/*
val ns: Rand[List[Int]] = for {
    x <- int
    y <- int
    xs <- ints(x)
} yield xs.map(_ % y)
*/

: 

### combinator - modify

In [112]:
def get[S]: State[S,S] = State(s => (s,s))

In [113]:
def set[S](s: S): State[S,Unit] = State(_ => ((),s))

In [114]:
def modify[S](f: S => S): State[S, Unit] = for {
    s <- get
    _ <- set(f(s))
} yield()

: 

# 결론:  side-effect 없이 프로그래밍하는 방법을 끊임없이 생각해봐라...